In [23]:
import sys
import sklearn
import numpy as np
import pandas as pd
from time import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import linear_model

f"Python: {sys.version}"

'Python: 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]'

### Setup

In [26]:
# trainData = pd.read_csv("tcdml1920-rec-click-pred--training.csv", index_col='recommendation_set_id', low_memory=False)
# testData = pd.read_csv("tcdml1920-rec-click-pred--test.csv", index_col='recommendation_set_id', low_memory=False)

trainData = pd.read_csv("tcdml1920-rec-click-pred--training.csv", low_memory=False)
testData = pd.read_csv("tcdml1920-rec-click-pred--test.csv", low_memory=False)
testData.dropna(how="all", inplace=True)

In [27]:
pd.set_option('display.max_columns', 50)
trainData.sample(5)

,recommendation_set_id,user_id,session_id,query_identifier,query_word_count,query_char_count,query_detected_language,query_document_id,document_language_provided,year_published,number_of_authors,abstract_word_count,abstract_char_count,abstract_detected_language,first_author_id,num_pubs_by_first_author,organization_id,application_type,item_type,request_received,hour_request_received,response_delivered,rec_processing_time,app_version,app_lang,user_os,user_os_version,user_java_version,user_timezone,country_by_ip,timezone_by_ip,local_time_of_request,local_hour_of_request,number_of_recs_in_set,recommendation_algorithm_id_used,algorithm_class,cbf_parser,search_title,search_keywords,search_abstract,time_recs_recieved,time_recs_displayed,time_recs_viewed,clicks,ctr,set_clicked
110962,173144,\N,\N,Withheld for privacy,14,97,en,\N,\N,\N,\N,\N,\N,\N,\N,\N,1,digital_library,academic_publication,05/04/2019 11:12,11,05/04/2019 11:12,6.102270,4.3.1,en,\N,\N,\N,\N,HU,200,05/04/2019 11:12,11,7,21,sentence_embeddings,\N,no,no,no,\N,\N,\N,0,0.0,0
289974,358679,\N,\N,Withheld for privacy,8,58,en,\N,\N,\N,\N,\N,\N,\N,\N,\N,1,digital_library,academic_publication,09/08/2019 00:57,0,09/08/2019 00:57,4.989180,4.3.1,en,\N,\N,\N,\N,US,-400,08/08/2019 18:57,18,7,21,sentence_embeddings,\N,no,no,no,\N,\N,\N,0,0.0,0
42947,91213,\N,\N,Withheld for privacy,7,46,en,11221656,en,2014,3,191,1190,en,3898825,231,1,digital_library,academic_publication,23/01/2019 05:25,5,23/01/2019 05:26,4.929330,4.3.1,en,\N,\N,\N,\N,AU,1030,23/01/2019 14:55,14,5,10,content_based_filtering,mlt_QP,yes,yes,yes,\N,\N,\N,0,0.0,0
118989,182839,\N,\N,Withheld for privacy,6,54,en,\N,\N,\N,\N,\N,\N,\N,\N,\N,1,digital_library,academic_publication,13/04/2019 21:11,21,13/04/2019 21:11,0.794035,4.3.1,en,\N,\N,\N,\N,EG,200,13/04/2019 21:11,21,7,12,content_based_filtering,edismax_QP,yes,yes,yes,\N,\N,\N,0,0.0,0
339358,410561,\N,\N,Withheld for privacy,12,117,en,\N,\N,\N,\N,\N,\N,\N,\N,\N,1,digital_library,academic_publication,12/09/2019 16:08,16,12/09/2019 16:08,11.803600,4.3.1,en,\N,\N,\N,\N,DE,200,12/09/2019 16:08,16,7,7,content_based_filtering,standard_QP,yes,yes,no,\N,\N,\N,0,0.0,0


In [28]:
trainData.replace(["\\N", "nA", "Not provided", "unknown", "*unknown*"], np.nan, inplace=True)
testData.replace(["\\N", "nA", "Not provided",  "unknown", "*unknown*"], np.nan, inplace=True)

In [29]:
# trainData.isnull().sum().sort_values()

In [30]:
numeric_columns = [
    "query_word_count",
    "query_char_count",
    "query_document_id",
    "year_published",
    "number_of_authors",
    "abstract_char_count",
    "abstract_word_count",
    "first_author_id",
    "num_pubs_by_first_author",
    "hour_request_received",
    "local_hour_of_request",
    "recommendation_algorithm_id_used",
    "clicks",
]

In [31]:
time_columns = [
    "request_received",
    "response_delivered",
    "local_time_of_request",
    "time_recs_recieved",
    "time_recs_displayed",
    "time_recs_viewed",
]

In [32]:
for col in numeric_columns:
    trainData[col] = pd.to_numeric(trainData[col], errors="coerce")
    testData[col] = pd.to_numeric(testData[col], errors="coerce")

In [33]:
for col in time_columns:
    trainData[col] = pd.to_datetime(trainData[col], format="%d/%m/%Y %H:%M")
    testData[col] = pd.to_datetime(testData[col], format="%d/%m/%Y %H:%M")

In [42]:
trainData.columns

Index(['recommendation_set_id', 'user_id', 'session_id', 'query_identifier',
       'query_word_count', 'query_char_count', 'query_detected_language',
       'query_document_id', 'document_language_provided', 'year_published',
       'number_of_authors', 'abstract_word_count', 'abstract_char_count',
       'abstract_detected_language', 'first_author_id',
       'num_pubs_by_first_author', 'organization_id', 'application_type',
       'item_type', 'request_received', 'hour_request_received',
       'response_delivered', 'rec_processing_time', 'app_version', 'app_lang',
       'user_os', 'user_os_version', 'user_java_version', 'user_timezone',
       'country_by_ip', 'timezone_by_ip', 'local_time_of_request',
       'local_hour_of_request', 'number_of_recs_in_set',
       'recommendation_algorithm_id_used', 'algorithm_class', 'cbf_parser',
       'search_title', 'search_keywords', 'search_abstract',
       'time_recs_recieved', 'time_recs_displayed', 'time_recs_viewed',
       'clicks', 

In [55]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50000)
trainData["query_document_id"].value_counts()

124015831.0    2004
124015835.0    1589
124015816.0    1490
123983263.0    1014
123983291.0     981
               ... 
11393216.0        1
123990331.0       1
12146070.0        1
12146053.0        1
16778331.0        1
Name: query_document_id, Length: 63780, dtype: int64

In [59]:
trainData.where( trainData['query_document_id'] == trainData['query_identifier'])

,recommendation_set_id,user_id,session_id,query_identifier,query_word_count,query_char_count,query_detected_language,query_document_id,document_language_provided,year_published,number_of_authors,abstract_word_count,abstract_char_count,abstract_detected_language,first_author_id,num_pubs_by_first_author,organization_id,application_type,item_type,request_received,hour_request_received,response_delivered,rec_processing_time,app_version,app_lang,user_os,user_os_version,user_java_version,user_timezone,country_by_ip,timezone_by_ip,local_time_of_request,local_hour_of_request,number_of_recs_in_set,recommendation_algorithm_id_used,algorithm_class,cbf_parser,search_title,search_keywords,search_abstract,time_recs_recieved,time_recs_displayed,time_recs_viewed,clicks,ctr,set_clicked
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN
385683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN
385684,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN
385685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN


In [50]:
# pd.options.display.float_format = 'describef}'.format
trainData["query_identifier"].describe()

count                   385533
unique                   30678
top       Withheld for privacy
freq                    270246
Name: query_identifier, dtype: object

In [21]:
# Row Duplication check TrainData
duplicateRowsTrain = trainData[trainData.duplicated(keep=False)]
duplicateRowsTrain.shape

(570, 45)

285 Duplicated train rows

In [18]:
# Row Duplication check TestData
duplicateRowsTest = testData[testData.duplicated()]
duplicateRowsTest.shape

(0, 45)

### Basic visualizations

In [ ]:
plt.figure(figsize=(24, 4))
# sns.countplot(x="query_identifier", data=train_set)
# plt.show()
sns.countplot(x="document_language_provided", data=train_set)
plt.show()
plt.figure(figsize=(24, 4))
sns.countplot(x="abstract_detected_language", data=train_set)
plt.show()
sns.countplot(x="application_type", data=train_set)
plt.show()
plt.figure(figsize=(24, 4))
sns.countplot(x="item_type", data=train_set)
plt.show()
plt.figure(figsize=(24, 4))
sns.countplot(x="app_version", data=train_set, order=pd.value_counts(
    train_set['app_version']).iloc[:10].index)
plt.show()
sns.countplot(x="app_lang", data=train_set)
plt.show()
sns.countplot(x="user_os", data=train_set)
plt.show()
# sns.countplot(x="user_os_version", data=train_set)
# plt.show()
# sns.countplot(x="user_java_version", data=train_set)
# plt.show()
plt.figure(figsize=(24, 4))
sns.countplot(x="user_timezone", data=train_set,
              order=pd.value_counts(train_set['user_timezone']).index)
plt.show()
plt.figure(figsize=(24, 4))
sns.countplot(x="country_by_ip", data=train_set, order=pd.value_counts(
    train_set['country_by_ip']).iloc[:15].index)
plt.show()
plt.figure(figsize=(24, 4))
sns.countplot(x="timezone_by_ip", data=train_set, order=pd.value_counts(
    train_set['timezone_by_ip']).iloc[:15].index)
plt.show()
plt.figure(figsize=(24, 4))
sns.countplot(x="algorithm_class", data=train_set)
plt.show()

### Encoding
- One hot
- Target

- Smote 
- Giving more weight to yes